In [21]:
import numpy as np
import h5py
import sys,os
from astropy.cosmology import Planck15 as cosmo, z_at_value

In [8]:
def construct_world_lines(isnap_start, isnap_end):

    # read snapshots A and B.
    snapdir ='/cosma6/data/dp004/dc-boot5/Ordered_Snapshots/Npart_512_Box_750-Fiducial/'
    
    NumLCPart = 0;
    NumLCPartTOT = 0;
    iLCfile = 0;

    # Initialise particle buffers
    PartBuff = np.empty((2,NumPartTOT), dtype = part)
    iobj = 0;
    
    # processes will read the snapshots and compute the lightcone crossings 
    for isnap in range(isnap_start, isnap_end):

        PartA = PartBuff[iobj] # point PartA to previous PartB
        headerOrderedA = headerOrderedB;
        iobj+=1;
        iobj %= 2;  #increment particle buffer index mod 2

        PartB = PartBuff[iobj] # and set PartB pointer to this address

        iexist = False;
        while (not iexist):
            input_fname = snapdir + 'ordered_snapshot.snap_{0:03d}.hdf5'.format(isnap)
            # Check that file actually exists for this snapshot
            iexist = os.path.exists(input_fname)
            if (not iexist):
                print('Snapshot file {0:03d} missing, skipping to next snapshot'.format(isnap))
                isnap+=1;

        # read the particle data
        iresult = load_ordered_subsnapshot(input_fname, PartBuff[iobj])
        headerOrderedB = headerOrdered;

        if (isnap>isnap_start):
            iworldline = get_world_lines(headerOrderedA.redshift, headerOrderedB.redshift);
            if(iworldline>0):
                print("Found {0:0d} world lines between snapshots {1:03d} and {2:03d}", NumLCPart, isnap-1,isnap);

    if(NumLCPart>0):
        # write out final files
        write_lightcone_data()
        print('{0:0d} particles written to lightcone', NumLCPart)

    print("Done: construct_world_lines")


In [16]:
def get_world_lines(zA, zB):
    # initialise worldline class for these two snapshots
    WL = WorldLine(zA,zB)
    WL.BoxSize = All.BoxSize
    WL.set_origin(LC.xobs, LC.yobs, LC.zobs)

    #test whether any of the particle worldlines could lie in the redshift range specified for the light cone
    if((LC.zmin > zA) | (LC.zmax < LC.zmin)):
        if DEBUG & (ThisTask ==0):
            print('LC.zmin; LC.zmax:= %14.7f\t%14.7f;\tzA:= %14.7f\t%14.7f\n'.format(LC.zmin,LC.zmax,zA,zB))
        return 0

    np = LC.nperiodic
        
    # loop over all particles in this batch
    for n in range(Npart+1):
        #particle A, B
        WL.set_A(PartA[n])
        WL.set_B(PartB[n])
  
        WL.calc_pos_mean_delta()
        WL.calc_vel_mean_delta()
        
        for ip in range(-np, np+1):
            for jp in range(-np, np+1):
                for kp in range(-np, np+1):
                    WL.set_offset(ip, jp, kp)
                    
                    icross = WL.test_lightcone_crossing
                    if(icross==1):
                        #find lightcone intersection
                        tLB=WL.get_lightcone_intersection();

                        #get the position of the particle at lightcone crossing
                        zzWL = WL.get_world_line(tLB)
                        
                        #test whether particles redshift is in the required range
                        if ((zzWL>=LC.zmin) & (zzWL<=LC.zmax)):
                            #add particle to the lightcone structure....
                            NumLCPart+=1       # increase local counter
                            NumLCPartTOT+=1    # increase global counter for LC particles

                            LCPart[NumLCPart].Pos = WL.Pos
                            LCPart[NumLCPart].Vel = WL.Vel
                            LCPart[NumLCPart].Id = PartA[n].Id
                            LCPart[NumLCPart].zz = WL.zz
                            LCPart[NumLCPart].rad = WL.rad
                            LCPart[NumLCPart].dec = WL.dec
                            LCPart[NumLCPart].ra  = WL.ra
    return NumLCPart

In [10]:
def EuclidDist(x1, y1, z1, x2, y2, z2):
    return np.sqrt((x1-x2)**2 + (y1-y2)**2 + (z1-z2)**2)

In [11]:
def EuclidDist2(pos1, pos2):
    return np.sqrt((pos1['x']-pos2['x'])**2 + (pos1['y']-pos2['y'])**2 + (pos1['z']-pos2['z'])**2)

In [14]:
class WorldLine:
    def __init__(self, zA, zB):
        # calculate lookback times for each redshift
        self.tLB_A = cosmo.lookback_time(zA)
        self.tLB_B = cosmo.lookback_time(zB)
        
        # compute comoving distance to each snapshot
        self.chiA = cosmo.comoving_distance(zA)
        self.chiB = cosmo.comoving_distance(zB)
        
        # compute the mean time and time difference between snapshots A and B
        self.Tbar = (tLB_A+tLB_B)/2.0
        self.delT = tLB_A-tLB_B
        
        self.BoxSize = 750 #default box size
        
    def set_origin(self, xoff, yoff, zoff):
        self.xoff0 = xoff * BoxSize
        self.yoff0 = yoff * BoxSize
        self.zoff0 = zoff * BoxSize
        
    def set_A(p):
        #positions
        xA = p.Pos[0]-xoff0
        yA = p.Pos[1]-yoff0
        zA = p.Pos[2]-zoff0
        #periodic(&xA,&yA,&zA)
    
        #velocities
        vxA = p.Vel[0]/sqrt(aA)  
        vyA = p.Vel[1]/sqrt(aA)
        vzA = p.Vel[2]/sqrt(aA)
        
    def set_B(p):
        #positions
        xB = p.Pos[0]-xoff0
        yB = p.Pos[1]-yoff0
        zB = p.Pos[2]-zoff0
        #periodic(&xA,&yA,&zA)
    
        #velocities
        vxB = p.Vel[0]/sqrt(aA)  
        vyB = p.Vel[1]/sqrt(aA)
        vzB = p.Vel[2]/sqrt(aA)
        
    def calc_pos_mean_delta(self):
        def mean_delta(a, b):
            delta = b-a
            if (abs(delta)>BoxSize/2.0):
                #Modify Part A as light is travelling from B->A
                if (delta > 0):
                    b-=BoxSize
                else:
                    b+=BoxSize   
            delta = b-a
            return (a+b)/2, delta
        
        self.xbar, self.delx = mean_delta(xA,xB)
        self.xbar, self.dely = mean_delta(yA,yB)
        self.zbar, self.delx = mean_delta(zA,zB)
    
    def calc_vel_mean_delta(self):
        def mean_delta(a, b):
            return (a+b)/2, b-a
        
        self.vxbar, self.delvx = mean_delta(vxA,vxB)
        self.vybar, self.delvy = mean_delta(vyA,vyB)
        self.vzbar, self.delzx = mean_delta(vzA,vzB)

    def set_offset(self, i, j, k):
        self.xoff = i * BoxSize
        self.yoff = j * BoxSize
        self.zoff = k * BoxSize
        
        # update mean position parameters with offset
        self.xbar = xbar + xoff
        self.ybar = ybar + yoff
        self.zbar = zbar + zoff
        
#%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%
    def test_lightcone_crossing():
        rA = np.sqrt((xA+xoff)**2 + (yA+yoff)**2 + (zA+zoff)**2)
        rB = np.sqrt((xB+xoff)**2 + (yB+yoff)**2 + (zB+zoff)**2)

        del1 = chiA-rA;
        del2 = chiB-rB;

        if((del1>=0.0) & (del2<=0.0)):
            iCROSS = 1;
        elif ((del1<= 0.0) & (del2>= 0.0)):
            iCROSS = 1;
        else:
            iCROSS = 0;

        return iCROSS;

#%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%
    def get_lightcone_intersection():
        # compute the lookback times to zA and zB
        tLB_MAX = tLB_A;
        tLB_MIN = tLB_B;

        epsi = 1.0e-2;
        RadDiff = 1.0e32;

        while ((abs(RadDiff)> epsi) & (icount<100)):
            # tLB to evaluate the lightcone condition
            tLB = 0.5*(tLB_MAX+tLB_MIN);

            #turn tLB into zz
            zz=z_at_value(cosmo.lookback_time)

            # compute the comoving geodesic distance to redshifts z(tLB)
            chi_com=cosmo.comoving_distance(zz)
        
            # compute the radial distance from the observer of the particle at this redshift using the world-line equation
            pos, vel = get_world_line(tLB_B)
        
            Rad = EuclidDist2(pos,obs_pos);
            RadDiff = chi_com-Rad;

            if(RadDiff > epsi):
                tLB_MAX = tLB;
            elif(RadDiff < -epsi):
                tLB_MIN = tLB;
            
             icount+=1

            if DEBUG:
                print("icount:= %3d;\ttLB:= %14.7f;\tzz:= %14.7f;\tchiLC:= %14.7f;\tRad:=%14.7f;\tRadDiff:=%14.7f\n"
                      .format(count,tLB,zz,chi_com,Rad,RadDiff))
        if(icount>100):
            print("could not find the intersection?????????");
        else:
            if DEBUG:
                print(" at: tLB:= %14.7f\n".format(tLB))
            
        return tLB;
    
#%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%
    def get_world_line(self,tLB):
        tt1 = (t-Tbar)/delT;
        tt2 = tt1*tt1;
        tt3 = tt2*tt1;

        def calc_WL(Xbar, delX, Vbar, delV):
            # calculate position and velocity: x and vx, using world line equations
            pos = Xbar-delV*delT/8.0 + (3*delX-Vbar*delT)*tt1/2.0 + delV*delT*tt2/2.0 - 2.0*(delX-Vbar*delT)*tt3
            pel = -0.5*Vbar + 1.5*delX/delT+delV*tt1 + 12.0*(Vbar-delX/delT)*tt2
            return pos,vel

        self.Pos = np.empty(1, dtype = vect)  
        self.Vel = np.empty(1, dtype = vect)  
        
        Pos['x'], Vel['x'] = calc_WL(WL.xbar, WL.delx, WL.vxbar, WL.delvx)
        Pos['y'], Vel['y'] = calc_WL(WL.ybar, WL.dely, WL.vybar, WL.delvy)
        Pos['z'], Vel['z'] = calc_WL(WL.zbar, WL.delz, WL.vzbar, WL.delvz)

        # calculate rad, ra , dec, and redshift for this worldline
        self.rad = EuclidDist2(Pos,obs_pos)       
        self.dec = np.rad2deg(np.arcsin(Pos['z']/rad))
        self.ra = np.rad2deg(np.arctan2(Pos['y'],Pos['x']))

        # lookup redshift corresponding to this r
        self.zz = z_at_value(cosmo.comoving_distance)

        return zz
    

In [18]:
class LightCone:
    def __init__(self):
        # set default lightcone parameters
        self.xobs = 0.5
        self.yobs = 0.5
        self.zobs = 0.5

        self.zmin = 1e-6
        self.zmax = 0.10
  
        self.chimin = 0.0
        self.chimax = 750.0
        self.fsky = 1.0

        self.NumLCPart = 0
        self.NumLCPartTOT = 2**27  

        self.nperiodic = 0

In [22]:
def get_periodic_replications(LC):
    # compute max and min distance of the lightcone
    LC.chimin = cosmo.comoving_distance(LC.zmin)
    LC.chimax = cosmo.comoving_distance(LC.zmax)

    # compute the number of half-boxlengths this corresponds to
    LC.nperiodic = 2*LC.chimax // LC.BoxSize

    print('Max comoving distance in lightcone:= {0:0f}'.format(LC.chimax))
    print('Min comoving distance in lightcone:= {0:0f}'.format(LC.chimin))
    print('BoxSize:= {0:0f}'.format(BoxSize))
    print('number of periodic replications required:= {0:0d}'.format(LC.nperiodic))
